In [ ]:
# -*- coding: utf-8
# Repórter Brasil (http://ruralometro.reporterbrasil.org.br/)
# Reinaldo Chaves (@paidatocandeira)
# Programa para encontrar quais deputados federais eleitos em 2014 receberam doações de empresas ou pessoas físicas que têm áreas embargadas pelo Ibama
# Também mostra quais deputados têm diretamente áreas embargadas pelo Ibama
#

In [1]:
import pandas as pd

In [2]:
import numpy as np

In [ ]:
# Arquivo de doações gerais para políticos brasileiros em 2014 - fonte TSE - acessado 25 de novembro de 2017

In [1]:
# Arquivos podem ser baixados aqui http://agencia.tse.jus.br/estatistica/sead/odsele/prestacao_contas/prestacao_final_2014.zip

In [3]:
xlsx = pd.ExcelFile('doacoes_25_11.xlsx')

In [5]:
cand_doacoes = pd.read_excel(xlsx, 'Planilha2', decimal = ",")

In [5]:
cand_doacoes = cand_doacoes[(cand_doacoes['Cargo'] == 'Deputado Federal')]

In [ ]:
# Reserva variável com os valores totais de doações para cada candidato a deputado federal 

In [15]:
total = cand_doacoes.groupby(['CPF_candidato', 'Nome_candidato', 'Cargo']).Valor.sum().reset_index()

In [16]:
total.columns = ['CPF_candidato', 'Nome_candidato', 'Cargo', 'Total_geral']

In [17]:
total = total[(total['Cargo'] == 'Deputado Federal')]

In [ ]:
# Exclui doações em branco ou zero - TSE afirma que proṕrios candidados registraram esses dados incorretamente

In [18]:
cand_doacoes = cand_doacoes[(cand_doacoes['CPF_CNPJ_doador'] != 0.0)]

In [19]:
cand_doacoes = cand_doacoes[pd.notnull(cand_doacoes['CPF_CNPJ_doador'])]

In [ ]:
# Carrega base com os deputados eleitos em 2014 - fonte TSE 

In [20]:
candidatos = pd.read_csv("candidatos_nome_urna_csv.csv",sep=';',encoding = 'latin_1')

In [21]:
eleitos = candidatos[(candidatos['Situacao'] == 'ELEITO POR QP') | (candidatos['Situacao'] == 'ELEITO POR MÉDIA') | (candidatos['Situacao'] == 'ELEITO')]

In [22]:
eleitos_d = eleitos[(eleitos['Cargo'] == 'DEPUTADO FEDERAL')]

In [ ]:
# Totais dos eleitos

In [23]:
eleitos_d_totais = pd.merge(eleitos_d, total, left_on='CPF', right_on='CPF_candidato')

In [ ]:
# Doações só dos eleitos

In [24]:
eleitos_d_doadores = pd.merge(cand_doacoes, eleitos_d, left_on='CPF_candidato', right_on='CPF')

In [25]:
eleitos_d_doadores.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47356 entries, 0 to 47355
Data columns (total 36 columns):
CNPJ_Prestador_Conta                 47356 non-null int64
Sequencial_Candidato                 47356 non-null int64
UF                                   47356 non-null object
Partido_eleicao                      47356 non-null object
Numero_candidato                     47356 non-null int64
Cargo_x                              47356 non-null object
Nome_candidato_x                     47356 non-null object
CPF_candidato                        47356 non-null int64
Recibo_Eleitoral                     47356 non-null object
Numero_documento                     47356 non-null object
CPF_CNPJ_doador                      47356 non-null float64
Nome_doador                          47356 non-null object
Nome_doador_Receita                  47356 non-null object
Sigla_UE_doador                      47356 non-null object
Numero_partido_doador                47356 non-null object
Numero_ca

In [ ]:
# Faz copia de coluna CNPJ e CPF para os futuras cruzamentos

In [26]:
eleitos_d_doadores['Operacoes1'] = eleitos_d_doadores['CPF_CNPJ_doador']

In [27]:
eleitos_d_doadores['Operacoes1'] = eleitos_d_doadores.Operacoes1.astype('int64')

In [ ]:
# Carrega base com empresas e pessoas físicas com áreas embargadas pelo Ibama. Fonte Ibama - acessado em 24 de novembro de 2017

In [1]:
# Arquivo pode ser baixado aqui https://servicos.ibama.gov.br/ctf/publico/areasembargadas/ConsultaPublicaAreasEmbargadas.php

In [28]:
ibama = pd.read_csv("ibama_24_11.csv",sep=';',encoding = 'latin_1')

In [ ]:
# Exclui informações em branco

In [29]:
ibama = ibama[pd.notnull(ibama['CPF_CNPJ_limpo'])]

In [30]:
ibama = ibama[(ibama['CPF_CNPJ_limpo'] != 0)]

In [31]:
ibama['Operacoes'] = ibama['CPF_CNPJ_limpo']

In [32]:
ibama['Operacoes'] = ibama.Operacoes.astype('int64')

In [33]:
ibama.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55463 entries, 0 to 55886
Data columns (total 18 columns):
N_TAD                            55463 non-null int64
SERIE_TAD                        55463 non-null object
Area_ha                          19649 non-null float64
AI                               53297 non-null float64
Nome_Razao_Social                55463 non-null object
CPF_CNPJ                         55463 non-null object
CPF_CNPJ_limpo                   55463 non-null float64
Localizacao                      49306 non-null object
UF_Embargo                       55463 non-null object
Municipio_Embargo                55463 non-null object
Código_IBGE_Municipio_Embargo    55463 non-null int64
Municipio                        55463 non-null object
Bairro                           53823 non-null object
Endereco                         55223 non-null object
Julgamento                       55463 non-null object
Infracao                         53297 non-null object
Data_Lista

In [ ]:
# Esse programa faz cruzamentos com o CNPJ raiz

In [ ]:
# O CNPJ, compreende as informações cadastrais das entidades de interesse das administrações tributárias da União, dos Estados, do Distrito Federal e dos Municípios, sendo que sua administração compete a Receita Federal do Brasil. O CNPJ é formado por 14 dígitos, onde os 8 primeiros números formam a base que identifica a empresa. (IN RFB 1634 de 06/05/2016)

In [ ]:
# O programa trabalha com CNPJs sem sinais, então considerados números. O problema aqui é que nos casos em que o CNPJ começa com zero o CNPJ raiz não será os 8 primeiros dígitos. Então para consultar o CNPJ raiz as bases precisam ser separadas de acordo com o tamanho numérico do CNPJ

In [ ]:
# Uma solução é usar o logaritmo de 10 para separar de acordo com o tamanho do campo. Usei o numpy.log10

In [ ]:
# Depois as bases separadas foram reunidas em um só arquivo, com uma nova coluna CNPJ_raiz para os cruzamentos

# Separa apenas embargos na lista que vieram de CNPJs - 14 dígitos¶

In [34]:
ibama_cnpjs14 = ibama[np.isin(np.log10(ibama.Operacoes.values).astype(int) + 1, [14])]

In [35]:
ibama_cnpjs14['CNPJ_raiz'] = ibama_cnpjs14['Operacoes'].astype(str).str[:8]

/home/reinaldo/Documentos/Code/RB/ibama/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [36]:
ibama_cnpjs14.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3825 entries, 24 to 55853
Data columns (total 19 columns):
N_TAD                            3825 non-null int64
SERIE_TAD                        3825 non-null object
Area_ha                          330 non-null float64
AI                               3654 non-null float64
Nome_Razao_Social                3825 non-null object
CPF_CNPJ                         3825 non-null object
CPF_CNPJ_limpo                   3825 non-null float64
Localizacao                      3518 non-null object
UF_Embargo                       3825 non-null object
Municipio_Embargo                3825 non-null object
Código_IBGE_Municipio_Embargo    3825 non-null int64
Municipio                        3825 non-null object
Bairro                           3766 non-null object
Endereco                         3806 non-null object
Julgamento                       3825 non-null object
Infracao                         3654 non-null object
Data_Lista                 

In [28]:
ibama_cnpjs14.to_csv('ibama_cnpjs14.csv')

# Separa apenas embargos na lista que vieram de CNPJs - 13 dígitos

In [37]:
ibama_cnpjs13 = ibama[np.isin(np.log10(ibama.Operacoes.values).astype(int) + 1, [13])]

In [38]:
ibama_cnpjs13['CNPJ_raiz'] = ibama_cnpjs13['Operacoes'].astype(str).str[:7]

/home/reinaldo/Documentos/Code/RB/ibama/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [39]:
ibama_cnpjs13.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5756 entries, 10 to 55833
Data columns (total 19 columns):
N_TAD                            5756 non-null int64
SERIE_TAD                        5756 non-null object
Area_ha                          460 non-null float64
AI                               5504 non-null float64
Nome_Razao_Social                5756 non-null object
CPF_CNPJ                         5756 non-null object
CPF_CNPJ_limpo                   5756 non-null float64
Localizacao                      5057 non-null object
UF_Embargo                       5756 non-null object
Municipio_Embargo                5756 non-null object
Código_IBGE_Municipio_Embargo    5756 non-null int64
Municipio                        5756 non-null object
Bairro                           5686 non-null object
Endereco                         5741 non-null object
Julgamento                       5756 non-null object
Infracao                         5504 non-null object
Data_Lista                 

In [32]:
ibama_cnpjs13.to_csv('ibama_cnpjs13.csv')

# Separa apenas embargos na lista que vieram de CNPJs - 12 dígitos

In [40]:
ibama_cnpjs12 = ibama[np.isin(np.log10(ibama.Operacoes.values).astype(int) + 1, [12])]

In [41]:
ibama_cnpjs12['CNPJ_raiz'] = ibama_cnpjs12['Operacoes'].astype(str).str[:6]

/home/reinaldo/Documentos/Code/RB/ibama/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [42]:
ibama_cnpjs12.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 327 entries, 33 to 55711
Data columns (total 19 columns):
N_TAD                            327 non-null int64
SERIE_TAD                        327 non-null object
Area_ha                          55 non-null float64
AI                               309 non-null float64
Nome_Razao_Social                327 non-null object
CPF_CNPJ                         327 non-null object
CPF_CNPJ_limpo                   327 non-null float64
Localizacao                      283 non-null object
UF_Embargo                       327 non-null object
Municipio_Embargo                327 non-null object
Código_IBGE_Municipio_Embargo    327 non-null int64
Municipio                        327 non-null object
Bairro                           321 non-null object
Endereco                         323 non-null object
Julgamento                       327 non-null object
Infracao                         309 non-null object
Data_Lista                       327 non-nul

In [36]:
ibama_cnpjs12.to_csv('ibama_cnpjs12.csv')

# Separa os CPFs dos embargos

In [43]:
ibama_cpfs = ibama[np.isin(np.log10(ibama.Operacoes.values).astype(int) + 1, [11, 10, 9, 8, 7, 6])]

In [44]:
ibama_cpfs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45555 entries, 0 to 55886
Data columns (total 18 columns):
N_TAD                            45555 non-null int64
SERIE_TAD                        45555 non-null object
Area_ha                          18804 non-null float64
AI                               43830 non-null float64
Nome_Razao_Social                45555 non-null object
CPF_CNPJ                         45555 non-null object
CPF_CNPJ_limpo                   45555 non-null float64
Localizacao                      40448 non-null object
UF_Embargo                       45555 non-null object
Municipio_Embargo                45555 non-null object
Código_IBGE_Municipio_Embargo    45555 non-null int64
Municipio                        45555 non-null object
Bairro                           44050 non-null object
Endereco                         45353 non-null object
Julgamento                       45555 non-null object
Infracao                         43830 non-null object
Data_Lista

In [39]:
ibama_cpfs.to_csv('ibama_cpfs.csv')

# Trabalha primeiro com as doações diretas

# Separa apenas doacoes diretas que vieram de CNPJs - 14 dígitos

In [45]:
doacoes_cnpjs14 = eleitos_d_doadores[np.isin(np.log10(eleitos_d_doadores.Operacoes1.values).astype(int) + 1, [14])]

In [46]:
doacoes_cnpjs14.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17649 entries, 0 to 47295
Data columns (total 37 columns):
CNPJ_Prestador_Conta                 17649 non-null int64
Sequencial_Candidato                 17649 non-null int64
UF                                   17649 non-null object
Partido_eleicao                      17649 non-null object
Numero_candidato                     17649 non-null int64
Cargo_x                              17649 non-null object
Nome_candidato_x                     17649 non-null object
CPF_candidato                        17649 non-null int64
Recibo_Eleitoral                     17649 non-null object
Numero_documento                     17649 non-null object
CPF_CNPJ_doador                      17649 non-null float64
Nome_doador                          17649 non-null object
Nome_doador_Receita                  17649 non-null object
Sigla_UE_doador                      17649 non-null object
Numero_partido_doador                17649 non-null object
Numero_ca

In [47]:
doacoes_cnpjs14['CNPJ_raiz'] = doacoes_cnpjs14['Operacoes1'].astype(str).str[:8]

/home/reinaldo/Documentos/Code/RB/ibama/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [43]:
doacoes_cnpjs14.to_csv('doacoes_cnpjs14.csv')

# Separa apenas doacoes diretas que vieram de CNPJs - 13 dígitos¶

In [48]:
doacoes_cnpjs13 = eleitos_d_doadores[np.isin(np.log10(eleitos_d_doadores.Operacoes1.values).astype(int) + 1, [13])]

In [49]:
doacoes_cnpjs13['CNPJ_raiz'] = doacoes_cnpjs13['Operacoes1'].astype(str).str[:7]

/home/reinaldo/Documentos/Code/RB/ibama/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [50]:
doacoes_cnpjs13.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5162 entries, 30 to 47355
Data columns (total 38 columns):
CNPJ_Prestador_Conta                 5162 non-null int64
Sequencial_Candidato                 5162 non-null int64
UF                                   5162 non-null object
Partido_eleicao                      5162 non-null object
Numero_candidato                     5162 non-null int64
Cargo_x                              5162 non-null object
Nome_candidato_x                     5162 non-null object
CPF_candidato                        5162 non-null int64
Recibo_Eleitoral                     5162 non-null object
Numero_documento                     5162 non-null object
CPF_CNPJ_doador                      5162 non-null float64
Nome_doador                          5162 non-null object
Nome_doador_Receita                  5162 non-null object
Sigla_UE_doador                      5162 non-null object
Numero_partido_doador                5162 non-null object
Numero_candidato_doador 

In [47]:
doacoes_cnpjs13.to_csv('doacoes_cnpjs13.csv')

# Separa apenas doacoes diretas que vieram de CNPJs - 12 dígitos¶

In [51]:
doacoes_cnpjs12 = eleitos_d_doadores[np.isin(np.log10(eleitos_d_doadores.Operacoes1.values).astype(int) + 1, [12])]

In [52]:
doacoes_cnpjs12['CNPJ_raiz'] = doacoes_cnpjs12['Operacoes1'].astype(str).str[:6]

/home/reinaldo/Documentos/Code/RB/ibama/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [53]:
doacoes_cnpjs12.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 955 entries, 169 to 45779
Data columns (total 38 columns):
CNPJ_Prestador_Conta                 955 non-null int64
Sequencial_Candidato                 955 non-null int64
UF                                   955 non-null object
Partido_eleicao                      955 non-null object
Numero_candidato                     955 non-null int64
Cargo_x                              955 non-null object
Nome_candidato_x                     955 non-null object
CPF_candidato                        955 non-null int64
Recibo_Eleitoral                     955 non-null object
Numero_documento                     955 non-null object
CPF_CNPJ_doador                      955 non-null float64
Nome_doador                          955 non-null object
Nome_doador_Receita                  955 non-null object
Sigla_UE_doador                      955 non-null object
Numero_partido_doador                955 non-null object
Numero_candidato_doador              95

In [51]:
doacoes_cnpjs12.to_csv('doacoes_cnpjs12.csv')

# Separa os CPFs de doacoes - documentos com 11 ou menos dígitos

In [54]:
doacoes_cpfs = eleitos_d_doadores[np.isin(np.log10(eleitos_d_doadores.Operacoes1.values).astype(int) + 1, [11, 10, 9, 8, 7, 6])]

In [55]:
doacoes_cpfs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23590 entries, 16 to 47344
Data columns (total 37 columns):
CNPJ_Prestador_Conta                 23590 non-null int64
Sequencial_Candidato                 23590 non-null int64
UF                                   23590 non-null object
Partido_eleicao                      23590 non-null object
Numero_candidato                     23590 non-null int64
Cargo_x                              23590 non-null object
Nome_candidato_x                     23590 non-null object
CPF_candidato                        23590 non-null int64
Recibo_Eleitoral                     23590 non-null object
Numero_documento                     23590 non-null object
CPF_CNPJ_doador                      23590 non-null float64
Nome_doador                          23590 non-null object
Nome_doador_Receita                  23590 non-null object
Sigla_UE_doador                      23590 non-null object
Numero_partido_doador                23590 non-null object
Numero_c

# Carrega base reunida de CNPJs de Ibama

In [56]:
ibama_cnpjs_final = pd.read_csv("ibama_cnpjs_28_11.csv",sep=',',encoding = 'utf_8')

In [57]:
ibama_cnpjs_final.reset_index()

,index,N_TAD,SERIE_TAD,Area_ha,AI,Nome_Razao_Social,CPF_CNPJ,CPF_CNPJ_limpo,Localizacao,UF_Embargo,Municipio_Embargo,Código_IBGE_Municipio_Embargo,Municipio,Bairro,Endereco,Julgamento,Infracao,Data_Lista,Operacoes,CNPJ_raiz
0,0,290136,C,NaN,464470.0,E. Souza de Oliveira&Cia Ltda - Me (Madereira ...,10.446.158/0001-43,10446158000143,E.M. DAS CHAGAS E CIA LTDA.,RO,Alto Paraíso,1100403,Alto Paraíso/RO,Perimetro Urbano,"Rodovia Br-421, S/N, Linha C-85 Km-278",07/02/2013,Exercer atividade potencialmente degradadora e...,17/4/09,10446158000143,10446158
1,1,587665,C,NaN,363364.0,Municipio de Italva,30.417.158/0001-22,30417158000122,Rod. Br 356 - Km 77 Bairro: Boa Vista,RJ,Italva,3302056,Italva/RJ,Boa Vista,Br 356 - Km 77,07/02/2017,"Construir, reformar, ampliar, instalar ou faze...",11/10/11,30417158000122,30417158
2,2,220910,C,NaN,299960.0,Neusa Garrido de Andrade - Me,10.765.568/0002-38,10765568000238,FAZENDA RIACHO FECHADO,PB,Coremas,2504801,Coremas/PB,Zona Rural,"Cruz da Tereza, S/N",pendente de julgamento,"Executar pesquisa, lavra ou extração de minera...",4/2/10,10765568000238,10765568
3,3,55254,A,NaN,138208.0,Prefeitura Municipal de Cachoeirinha,87.990.800/0001-85,87990800000185,"DEPOSITO DE LIXO A CEU ABERTO , SEM LICENCIAME...",RS,Porto Alegre,4314902,Cachoeirinha/RS,Centro,"Av. Flores da Cunha, 2251",10/04/1997,Exercer atividade potencialmente degradadora s...,19/3/07,87990800000185,87990800
4,4,238061,C,NaN,484500.0,3 de Julho Ind. e Com. de Mat. para Construção...,36.870.947/0001-09,36870947000109,Loteamento Petrópolis - São Miguel do Araguaia...,GO,São Miguel do Araguaia,5220207,São Miguel do Araguaia/GO,Jardim Petrópolis,Av: Pioneira Qd. 27 Lt. 10,pendente de julgamento,"Executar pesquisa, lavra ou extração de minera...",8/10/12,36870947000109,36870947
5,5,719718,E,NaN,9113257.0,4 a Comércio e Transportes de Madeiras Ltda - Me,17.769.395/0001-94,17769395000194,"Rua: G, n° 524 - Lote 12 - Quadra 03 - Setor I...",MT,Aripuanã,5101407,Aripuanã/MT,Setor Industrial,Rua G,pendente de julgamento,Infração da Flora(Não Classificada-Móvel),21/2/17,17769395000194,17769395
6,6,94369,C,NaN,724983.0,A & D Industria e Comercio de Madeiras Ltda,10.783.053/0001-80,10783053000180,Pátio da empresa.,RO,Espigão D'Oeste,1100098,Espigão D'Oeste/RO,Setor Industrial,"Rodovia Br 387 Km 2,5 S/N",pendente de julgamento,"Elaborar ou apresentar informação, estudo, lau...",7/11/13,10783053000180,10783053
7,7,439832,C,NaN,483168.0,A & L Ferreira Ltda - Me,25.133.992/0001-56,25133992000156,NaN,GO,Trindade,5221403,Trindade/GO,Setor Central,"Avenida Manoel Monteiro, 1893, Quadra 74 Lote ...",pendente de julgamento,"Construir, reformar, ampliar, instalar ou faze...",27/10/11,25133992000156,25133992
8,8,685458,E,NaN,9075919.0,A Alves de Souza Industria - Me,17.669.523/0001-28,17669523000128,A Alves de Souza Indústria ME.,PA,Altamira,1500602,Altamira/PA,Cachoeira da Serra,Av Rio Madeira - Setor Industrial - Lote 16,pendente de julgamento,Infração da Flora(Não Classificada-Móvel),4/8/16,17669523000128,17669523
9,9,33907,E,NaN,9106889.0,A Alves de Souza Industria - Me,17.669.523/0001-28,17669523000128,RODOVIA BR 163 KM 121 - AV RIO MADEIRAS - SETO...,PA,Altamira,1500602,Altamira/PA,Cachoeira da Serra,Av Rio Madeira - Setor Industrial - Lote 16,pendente de julgamento,Infração da Flora(Não Classificada-Móvel),12/9/16,17669523000128,17669523


# Carrega base reunida de CNPJs de doacoes diretas

In [58]:
doacoes_cnpjs_final = pd.read_csv("doacoes_cnpjs_reunidos.csv",sep=',',encoding = 'utf_8', decimal = ",")

In [59]:
doacoes_cnpjs_final.reset_index()

,index,CNPJ_Prestador_Conta,Sequencial_Candidato,UF,Partido_eleicao,Numero_candidato,Cargo_x,Nome_candidato_x,CPF_candidato,Recibo_Eleitoral,...,Nome_doador_originario_Receita,Estado,Cargo_y,Nome_candidato_y,CPF,Nome_urna,Partido,Situacao,Operacoes,CNPJ_raiz
0,0,20567761000137,50000000670,BA,PRB,1012,Deputado Federal,ERONILDES VASCONCELOS CARVALHO,66858933549,010120600000BA000035,...,CARMEN LUCIA MEIRA MAGALHAES,BAHIA,DEPUTADO FEDERAL,ERONILDES VASCONCELOS CARVALHO,66858933549,TIA ERON,PRB,ELEITO POR QP,20566882000164,20566882
1,1,20567761000137,50000000670,BA,PRB,1012,Deputado Federal,ERONILDES VASCONCELOS CARVALHO,66858933549,010120600000BA000027,...,GERALDO NASCIMENTO SANTOS,BAHIA,DEPUTADO FEDERAL,ERONILDES VASCONCELOS CARVALHO,66858933549,TIA ERON,PRB,ELEITO POR QP,20566882000164,20566882
2,2,20567761000137,50000000670,BA,PRB,1012,Deputado Federal,ERONILDES VASCONCELOS CARVALHO,66858933549,010120600000BA000028,...,VALTER LUCAS PEREIRA,BAHIA,DEPUTADO FEDERAL,ERONILDES VASCONCELOS CARVALHO,66858933549,TIA ERON,PRB,ELEITO POR QP,20566882000164,20566882
3,3,20567761000137,50000000670,BA,PRB,1012,Deputado Federal,ERONILDES VASCONCELOS CARVALHO,66858933549,010120600000BA000025,...,VALTER LUCAS PEREIRA,BAHIA,DEPUTADO FEDERAL,ERONILDES VASCONCELOS CARVALHO,66858933549,TIA ERON,PRB,ELEITO POR QP,20566882000164,20566882
4,4,20567761000137,50000000670,BA,PRB,1012,Deputado Federal,ERONILDES VASCONCELOS CARVALHO,66858933549,010120600000BA000031,...,ISNARD PIMENTA DE ARAUJO,BAHIA,DEPUTADO FEDERAL,ERONILDES VASCONCELOS CARVALHO,66858933549,TIA ERON,PRB,ELEITO POR QP,20566882000164,20566882
5,5,20567761000137,50000000670,BA,PRB,1012,Deputado Federal,ERONILDES VASCONCELOS CARVALHO,66858933549,010120600000BA000026,...,NEIALBERT NASCIMENTO FERREIRA,BAHIA,DEPUTADO FEDERAL,ERONILDES VASCONCELOS CARVALHO,66858933549,TIA ERON,PRB,ELEITO POR QP,20566882000164,20566882
6,6,20567761000137,50000000670,BA,PRB,1012,Deputado Federal,ERONILDES VASCONCELOS CARVALHO,66858933549,010120600000BA000014,...,CARLOS ALBERTO DOS SANTOS FERREIRA,BAHIA,DEPUTADO FEDERAL,ERONILDES VASCONCELOS CARVALHO,66858933549,TIA ERON,PRB,ELEITO POR QP,20566882000164,20566882
7,7,20567761000137,50000000670,BA,PRB,1012,Deputado Federal,ERONILDES VASCONCELOS CARVALHO,66858933549,010120600000BA000015,...,CARLOS ALBERTO DOS SANTOS FERREIRA,BAHIA,DEPUTADO FEDERAL,ERONILDES VASCONCELOS CARVALHO,66858933549,TIA ERON,PRB,ELEITO POR QP,20566882000164,20566882
8,8,20567761000137,50000000670,BA,PRB,1012,Deputado Federal,ERONILDES VASCONCELOS CARVALHO,66858933549,010120600000BA000011,...,MARIA CELIA SOUZA DE ALMEIDA,BAHIA,DEPUTADO FEDERAL,ERONILDES VASCONCELOS CARVALHO,66858933549,TIA ERON,PRB,ELEITO POR QP,20566882000164,20566882
9,9,20567761000137,50000000670,BA,PRB,1012,Deputado Federal,ERONILDES VASCONCELOS CARVALHO,66858933549,010120600000BA000010,...,MARIA CELIA SOUZA DE ALMEIDA,BAHIA,DEPUTADO FEDERAL,ERONILDES VASCONCELOS CARVALHO,66858933549,TIA ERON,PRB,ELEITO POR QP,20566882000164,20566882


# Cruzamento 1 - de CNPJs - candidatos eleitos e doacoes diretas + nomes em embargos do Ibama

In [60]:
ibama_doadores_diretos = pd.merge(doacoes_cnpjs_final, ibama_cnpjs_final.drop_duplicates('CNPJ_raiz'), left_on='CNPJ_raiz', right_on='CNPJ_raiz')

In [61]:
ibama_doadores_diretos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 210 entries, 0 to 209
Data columns (total 56 columns):
CNPJ_Prestador_Conta                 210 non-null int64
Sequencial_Candidato                 210 non-null int64
UF                                   210 non-null object
Partido_eleicao                      210 non-null object
Numero_candidato                     210 non-null int64
Cargo_x                              210 non-null object
Nome_candidato_x                     210 non-null object
CPF_candidato                        210 non-null int64
Recibo_Eleitoral                     210 non-null object
Numero_documento                     210 non-null object
CPF_CNPJ_doador                      210 non-null int64
Nome_doador                          210 non-null object
Nome_doador_Receita                  210 non-null object
Sigla_UE_doador                      210 non-null object
Numero_partido_doador                210 non-null object
Numero_candidato_doador              210 non-

In [61]:
ibama_doadores_diretos.to_csv('ibama_doadores_diretos.csv')

In [62]:
# Pega total dessas doações em valores por deputado

In [63]:
total_ibama_diretos_cnpj = ibama_doadores_diretos.groupby(['CPF_candidato', 'Nome_candidato_x', 'Cargo_x']).Valor.sum().reset_index()

In [64]:
total_ibama_diretos_cnpj.columns = ['CPF_candidato', 'Nome_candidato_x', 'Cargo_x', 'Total_doacoes_diretas_com_CNPJs']

In [65]:
total_ibama_diretos_cnpj.reset_index()

,index,CPF_candidato,Nome_candidato_x,Cargo_x,Total_doacoes_diretas_com_CNPJs
0,0,105457434,JARBAS DE ANDRADE VASCONCELOS,Deputado Federal,50000.00
1,1,398099863,CARLOS ALBERTO ROLIM ZARATTINI,Deputado Federal,365000.04
2,2,564834904,NELSON MEURER,Deputado Federal,4550.00
3,3,590048783,RODRIGO FELINTO IBARRA EPITÁCIO MAIA,Deputado Federal,75000.00
4,4,770338410,EFRAIM DE ARAÚJO MORAIS FILHO,Deputado Federal,80000.00
5,5,1117690199,JOÃO HENRIQUE HOLANDA CALDAS,Deputado Federal,110000.00
6,6,1266637699,NEWTON CARDOSO JÚNIOR,Deputado Federal,50000.00
7,7,1311846794,WASHINGTON REIS DE OLIVEIRA,Deputado Federal,100000.00
8,8,1338190601,CAIO NARCIO RODRIGUES DA SILVEIRA,Deputado Federal,100000.00
9,9,1416576770,ALESSANDRO LUCCIOLA MOLON,Deputado Federal,50000.00


# Trabalha agora com as doações indiretas

In [66]:
eleitos_d_doadores = eleitos_d_doadores[(eleitos_d_doadores['CPF_CNPJ_doador_originario'] != '#NULO')] 

In [67]:
eleitos_d_doadores['Operacoes2'] = eleitos_d_doadores['CPF_CNPJ_doador_originario']

In [68]:
eleitos_d_doadores['Operacoes2'] = eleitos_d_doadores.Operacoes2.astype('int64')

In [69]:
eleitos_d_doadores.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10323 entries, 0 to 47347
Data columns (total 38 columns):
CNPJ_Prestador_Conta                 10323 non-null int64
Sequencial_Candidato                 10323 non-null int64
UF                                   10323 non-null object
Partido_eleicao                      10323 non-null object
Numero_candidato                     10323 non-null int64
Cargo_x                              10323 non-null object
Nome_candidato_x                     10323 non-null object
CPF_candidato                        10323 non-null int64
Recibo_Eleitoral                     10323 non-null object
Numero_documento                     10323 non-null object
CPF_CNPJ_doador                      10323 non-null float64
Nome_doador                          10323 non-null object
Nome_doador_Receita                  10323 non-null object
Sigla_UE_doador                      10323 non-null object
Numero_partido_doador                10323 non-null object
Numero_ca

# Separa apenas doacoes indiretas que vieram de CNPJs - 14 dígitos

In [70]:
doacoes_cnpjs14_orig = eleitos_d_doadores[np.isin(np.log10(eleitos_d_doadores.Operacoes2.values).astype(int) + 1, [14])]

In [71]:
doacoes_cnpjs14_orig['CNPJ_raiz'] = doacoes_cnpjs14_orig['Operacoes2'].astype(str).str[:8]

/home/reinaldo/Documentos/Code/RB/ibama/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [72]:
doacoes_cnpjs14_orig.to_csv('doacoes_orig_cnpjs14.csv')

In [73]:
doacoes_cnpjs14_orig.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5615 entries, 30 to 47347
Data columns (total 39 columns):
CNPJ_Prestador_Conta                 5615 non-null int64
Sequencial_Candidato                 5615 non-null int64
UF                                   5615 non-null object
Partido_eleicao                      5615 non-null object
Numero_candidato                     5615 non-null int64
Cargo_x                              5615 non-null object
Nome_candidato_x                     5615 non-null object
CPF_candidato                        5615 non-null int64
Recibo_Eleitoral                     5615 non-null object
Numero_documento                     5615 non-null object
CPF_CNPJ_doador                      5615 non-null float64
Nome_doador                          5615 non-null object
Nome_doador_Receita                  5615 non-null object
Sigla_UE_doador                      5615 non-null object
Numero_partido_doador                5615 non-null object
Numero_candidato_doador 

# Separa apenas doacoes indiretas que vieram de CNPJs - 13 dígitos

In [74]:
doacoes_cnpjs13_orig = eleitos_d_doadores[np.isin(np.log10(eleitos_d_doadores.Operacoes2.values).astype(int) + 1, [13])]

In [75]:
doacoes_cnpjs13_orig['CNPJ_raiz'] = doacoes_cnpjs13_orig['Operacoes2'].astype(str).str[:7]

/home/reinaldo/Documentos/Code/RB/ibama/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [76]:
doacoes_cnpjs13_orig.to_csv('doacoes_orig_cnpjs13.csv')

In [77]:
doacoes_cnpjs13_orig.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2617 entries, 54 to 47346
Data columns (total 39 columns):
CNPJ_Prestador_Conta                 2617 non-null int64
Sequencial_Candidato                 2617 non-null int64
UF                                   2617 non-null object
Partido_eleicao                      2617 non-null object
Numero_candidato                     2617 non-null int64
Cargo_x                              2617 non-null object
Nome_candidato_x                     2617 non-null object
CPF_candidato                        2617 non-null int64
Recibo_Eleitoral                     2617 non-null object
Numero_documento                     2617 non-null object
CPF_CNPJ_doador                      2617 non-null float64
Nome_doador                          2617 non-null object
Nome_doador_Receita                  2617 non-null object
Sigla_UE_doador                      2617 non-null object
Numero_partido_doador                2617 non-null object
Numero_candidato_doador 

# Separa apenas doacoes indiretas que vieram de CNPJs - 12 dígitos

In [78]:
doacoes_cnpjs12_orig = eleitos_d_doadores[np.isin(np.log10(eleitos_d_doadores.Operacoes2.values).astype(int) + 1, [12])]

In [79]:
doacoes_cnpjs12_orig['CNPJ_raiz'] = doacoes_cnpjs12_orig['Operacoes2'].astype(str).str[:6]

/home/reinaldo/Documentos/Code/RB/ibama/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [80]:
doacoes_cnpjs12_orig.to_csv('doacoes_orig_cnpjs12.csv')

In [81]:
doacoes_cnpjs12_orig.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 243 entries, 413 to 47141
Data columns (total 39 columns):
CNPJ_Prestador_Conta                 243 non-null int64
Sequencial_Candidato                 243 non-null int64
UF                                   243 non-null object
Partido_eleicao                      243 non-null object
Numero_candidato                     243 non-null int64
Cargo_x                              243 non-null object
Nome_candidato_x                     243 non-null object
CPF_candidato                        243 non-null int64
Recibo_Eleitoral                     243 non-null object
Numero_documento                     243 non-null object
CPF_CNPJ_doador                      243 non-null float64
Nome_doador                          243 non-null object
Nome_doador_Receita                  243 non-null object
Sigla_UE_doador                      243 non-null object
Numero_partido_doador                243 non-null object
Numero_candidato_doador              24

# Separa doacoes indiretas com CPFs

In [82]:
doacoes_cpfs_orig = eleitos_d_doadores[np.isin(np.log10(eleitos_d_doadores.Operacoes2.values).astype(int) + 1, [11, 10, 9, 8, 7, 6])] 

In [83]:
doacoes_cpfs_orig.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1848 entries, 0 to 47108
Data columns (total 38 columns):
CNPJ_Prestador_Conta                 1848 non-null int64
Sequencial_Candidato                 1848 non-null int64
UF                                   1848 non-null object
Partido_eleicao                      1848 non-null object
Numero_candidato                     1848 non-null int64
Cargo_x                              1848 non-null object
Nome_candidato_x                     1848 non-null object
CPF_candidato                        1848 non-null int64
Recibo_Eleitoral                     1848 non-null object
Numero_documento                     1848 non-null object
CPF_CNPJ_doador                      1848 non-null float64
Nome_doador                          1848 non-null object
Nome_doador_Receita                  1848 non-null object
Sigla_UE_doador                      1848 non-null object
Numero_partido_doador                1848 non-null object
Numero_candidato_doador  

# Carrega base reunida de CNPJs de doacoes indiretas

In [84]:
doacoes_cnpjs_final_orig = pd.read_csv("doacoes_cnpjs_reunidos_orig.csv",sep=',',encoding = 'utf_8', decimal = ',')

# Cruzamento 2 - de CNPJs - candidatos eleitos e doacoes indiretas + nomes em embargos do Ibama

In [85]:
ibama_doadores_indiretos = pd.merge(doacoes_cnpjs_final_orig, ibama_cnpjs_final.drop_duplicates('CNPJ_raiz'), left_on='CNPJ_raiz', right_on='CNPJ_raiz')

In [86]:
ibama_doadores_indiretos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 941 entries, 0 to 940
Data columns (total 57 columns):
CNPJ_Prestador_Conta                 941 non-null int64
Sequencial_Candidato                 941 non-null int64
UF                                   941 non-null object
Partido_eleicao                      941 non-null object
Numero_candidato                     941 non-null int64
Cargo_x                              941 non-null object
Nome_candidato_x                     941 non-null object
CPF_candidato                        941 non-null int64
Recibo_Eleitoral                     941 non-null object
Numero_documento                     941 non-null object
CPF_CNPJ_doador                      941 non-null int64
Nome_doador                          941 non-null object
Nome_doador_Receita                  941 non-null object
Sigla_UE_doador                      941 non-null object
Numero_partido_doador                941 non-null object
Numero_candidato_doador              941 non-

In [90]:
ibama_doadores_indiretos.to_csv('ibama_doadores_indiretos.csv')

In [87]:
# Pega total dessas doações em valores por deputado

In [88]:
total_ibama_indiretos_cnpj = ibama_doadores_indiretos.groupby(['CPF_candidato', 'Nome_candidato_x', 'Cargo_x']).Valor.sum().reset_index()

In [89]:
total_ibama_indiretos_cnpj.columns = ['CPF_candidato', 'Nome_candidato_x', 'Cargo_x', 'Total_doacoes_indiretas_com_CNPJs']

In [90]:
total_ibama_indiretos_cnpj.reset_index()

,index,CPF_candidato,Nome_candidato_x,Cargo_x,Total_doacoes_indiretas_com_CNPJs
0,0,31226540,MÁRIO SÍLVIO MENDES NEGROMONTE JÚNIOR,Deputado Federal,116948.64
1,1,262905701,PEDRO PAULO CARVALHO TEIXEIRA,Deputado Federal,150000.00
2,2,263382893,DAGOBERTO NOGUEIRA FILHO,Deputado Federal,300000.00
3,3,309745187,JOSÉ FRANCISCO PAES LANDIM,Deputado Federal,100000.00
4,4,352748672,BONIFÁCIO JOSÉ TAMM DE ANDRADA,Deputado Federal,125000.00
5,5,398099863,CARLOS ALBERTO ROLIM ZARATTINI,Deputado Federal,545500.01
6,6,405819773,ANTONIO PEDRO INDIO DA COSTA,Deputado Federal,306000.00
7,7,445689676,RODRIGO OTAVIO SOARES PACHECO,Deputado Federal,1461.33
8,8,502302801,SERGIO OLIMPIO GOMES,Deputado Federal,136239.75
9,9,551632895,HERCULANO CASTILHO PASSOS JUNIOR,Deputado Federal,10187.17


# Cruzamento 3 - de CPFs - candidatos eleitos e doacoes diretas + nomes na lista de embargos do Ibama

In [91]:
ibama_doadores_diretos_cpf = pd.merge(doacoes_cpfs, ibama_cpfs.drop_duplicates('CPF_CNPJ_limpo'), left_on='CPF_CNPJ_doador', right_on='CPF_CNPJ_limpo')

In [92]:
ibama_doadores_diretos_cpf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 102 entries, 0 to 101
Data columns (total 55 columns):
CNPJ_Prestador_Conta                 102 non-null int64
Sequencial_Candidato                 102 non-null int64
UF                                   102 non-null object
Partido_eleicao                      102 non-null object
Numero_candidato                     102 non-null int64
Cargo_x                              102 non-null object
Nome_candidato_x                     102 non-null object
CPF_candidato                        102 non-null int64
Recibo_Eleitoral                     102 non-null object
Numero_documento                     102 non-null object
CPF_CNPJ_doador                      102 non-null float64
Nome_doador                          102 non-null object
Nome_doador_Receita                  102 non-null object
Sigla_UE_doador                      102 non-null object
Numero_partido_doador                102 non-null object
Numero_candidato_doador              102 no

In [100]:
ibama_doadores_diretos_cpf.to_csv('ibama_doadores_diretos_cpf.csv')

In [93]:
# Pega total dessas doações em valores por deputado

In [94]:
total_ibama_diretos_cpf = ibama_doadores_diretos_cpf.groupby(['CPF_candidato', 'Nome_candidato_x', 'Cargo_x']).Valor.sum().reset_index()

In [95]:
total_ibama_diretos_cpf.columns = ['CPF_candidato', 'Nome_candidato_x', 'Cargo_x', 'Total_doacoes_diretas_com_CPFs']

In [96]:
total_ibama_diretos_cpf.reset_index()

,index,CPF_candidato,Nome_candidato_x,Cargo_x,Total_doacoes_diretas_com_CPFs
0,0,352748672,BONIFÁCIO JOSÉ TAMM DE ANDRADA,Deputado Federal,17127.00
1,1,1782037500,JOSÉ CARLOS ALELUIA COSTA,Deputado Federal,1000.00
2,2,1960970453,LUIZ GONZAGA PATRIOTA,Deputado Federal,63600.00
3,3,2147279129,JOSE WILSON SANTIAGO FILHO,Deputado Federal,500.00
4,4,2441369806,LUIZ PAULO TEIXEIRA FERREIRA,Deputado Federal,1500.00
5,5,2875900200,NILSON PINTO DE OLIVEIRA,Deputado Federal,100000.00
6,6,3098871946,JOSÉ CARLOS BECKER DE OLIVEIRA E SILVA,Deputado Federal,21000.00
7,7,4707960812,EZEQUIEL ANGELO FONSECA,Deputado Federal,237980.00
8,8,4789113515,JOSÉ ALVES ROCHA,Deputado Federal,114000.00
9,9,8584060120,CESAR HANNA HALUM,Deputado Federal,7000.00


# Cruzamento 4 - de CPFs - candidatos eleitos e doacoes indiretas + nomes em embargos do Ibama

In [97]:
doacoes_cpfs_orig = doacoes_cpfs_orig[(doacoes_cpfs_orig['CPF_CNPJ_doador_originario'] != '#NULO')] 

In [98]:
ibama_doadores_indiretos_cpf = pd.merge(doacoes_cpfs_orig, ibama_cpfs.drop_duplicates('CPF_CNPJ_limpo'), left_on='CPF_CNPJ_doador_originario', right_on='CPF_CNPJ_limpo')

In [99]:
ibama_doadores_indiretos_cpf.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 56 columns):
CNPJ_Prestador_Conta                 0 non-null int64
Sequencial_Candidato                 0 non-null int64
UF                                   0 non-null object
Partido_eleicao                      0 non-null object
Numero_candidato                     0 non-null int64
Cargo_x                              0 non-null object
Nome_candidato_x                     0 non-null object
CPF_candidato                        0 non-null int64
Recibo_Eleitoral                     0 non-null object
Numero_documento                     0 non-null object
CPF_CNPJ_doador                      0 non-null float64
Nome_doador                          0 non-null object
Nome_doador_Receita                  0 non-null object
Sigla_UE_doador                      0 non-null object
Numero_partido_doador                0 non-null object
Numero_candidato_doador              0 non-null int64
Cod_setor_economico_doador          

# Cruzamento 5 - candidatos eleitos que estão na lista de embargos do Ibama

In [100]:
eleitos_d_ibama = pd.merge(ibama_cpfs, eleitos_d, left_on='CPF_CNPJ_limpo', right_on='CPF')

In [101]:
eleitos_d_ibama.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9 entries, 0 to 8
Data columns (total 25 columns):
N_TAD                            9 non-null int64
SERIE_TAD                        9 non-null object
Area_ha                          5 non-null float64
AI                               9 non-null float64
Nome_Razao_Social                9 non-null object
CPF_CNPJ                         9 non-null object
CPF_CNPJ_limpo                   9 non-null object
Localizacao                      9 non-null object
UF_Embargo                       9 non-null object
Municipio_Embargo                9 non-null object
Código_IBGE_Municipio_Embargo    9 non-null int64
Municipio                        9 non-null object
Bairro                           9 non-null object
Endereco                         9 non-null object
Julgamento                       9 non-null object
Infracao                         9 non-null object
Data_Lista                       9 non-null object
Operacoes                       

In [111]:
eleitos_d_ibama.to_csv('ibama_politicos_eleitos.csv')

# Cria arquivo com totais

In [102]:
soma_total_inicial1 = pd.merge(total_ibama_diretos_cnpj, total_ibama_indiretos_cnpj, left_on='CPF_candidato', right_on='CPF_candidato', how='outer')

In [103]:
soma_total_inicial2 = pd.merge(soma_total_inicial1, total_ibama_diretos_cpf, left_on='CPF_candidato', right_on='CPF_candidato', how='outer')

In [104]:
soma_total_inicial2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 294 entries, 0 to 293
Data columns (total 10 columns):
CPF_candidato                        294 non-null int64
Nome_candidato_x_x                   120 non-null object
Cargo_x_x                            120 non-null object
Total_doacoes_diretas_com_CNPJs      120 non-null float64
Nome_candidato_x_y                   227 non-null object
Cargo_x_y                            227 non-null object
Total_doacoes_indiretas_com_CNPJs    227 non-null float64
Nome_candidato_x                     36 non-null object
Cargo_x                              36 non-null object
Total_doacoes_diretas_com_CPFs       36 non-null float64
dtypes: float64(3), int64(1), object(6)
memory usage: 25.3+ KB


In [105]:
soma_total_final = pd.merge(soma_total_inicial2, eleitos_d_totais, left_on='CPF_candidato', right_on='CPF_candidato')

In [106]:
soma_total_final.reset_index()

,index,CPF_candidato,Nome_candidato_x_x,Cargo_x_x,Total_doacoes_diretas_com_CNPJs,Nome_candidato_x_y,Cargo_x_y,Total_doacoes_indiretas_com_CNPJs,Nome_candidato_x_x,Cargo_x_x,...,Estado,Cargo_x_y,Nome_candidato_x_y,CPF,Nome_urna,Partido,Situacao,Nome_candidato_y,Cargo_y,Total_geral
0,0,105457434,JARBAS DE ANDRADE VASCONCELOS,Deputado Federal,50000.00,NaN,NaN,NaN,NaN,NaN,...,PERNAMBUCO,DEPUTADO FEDERAL,JARBAS DE ANDRADE VASCONCELOS,105457434,JARBAS,PMDB,ELEITO POR QP,JARBAS DE ANDRADE VASCONCELOS,Deputado Federal,1979036.00
1,1,398099863,CARLOS ALBERTO ROLIM ZARATTINI,Deputado Federal,365000.04,CARLOS ALBERTO ROLIM ZARATTINI,Deputado Federal,545500.01,NaN,NaN,...,SÃO PAULO,DEPUTADO FEDERAL,CARLOS ALBERTO ROLIM ZARATTINI,398099863,ZARATTINI,PT,ELEITO POR QP,CARLOS ALBERTO ROLIM ZARATTINI,Deputado Federal,6244292.85
2,2,564834904,NELSON MEURER,Deputado Federal,4550.00,NELSON MEURER,Deputado Federal,1217000.00,NaN,NaN,...,PARANÁ,DEPUTADO FEDERAL,NELSON MEURER,564834904,NELSON MEURER,PP,ELEITO POR QP,NELSON MEURER,Deputado Federal,2645682.20
3,3,590048783,RODRIGO FELINTO IBARRA EPITÁCIO MAIA,Deputado Federal,75000.00,NaN,NaN,NaN,NaN,NaN,...,RIO DE JANEIRO,DEPUTADO FEDERAL,RODRIGO FELINTO IBARRA EPITÁCIO MAIA,590048783,RODRIGO MAIA,DEM,ELEITO POR QP,RODRIGO FELINTO IBARRA EPITÁCIO MAIA,Deputado Federal,2356183.05
4,4,770338410,EFRAIM DE ARAÚJO MORAIS FILHO,Deputado Federal,80000.00,NaN,NaN,NaN,NaN,NaN,...,PARAÍBA,DEPUTADO FEDERAL,EFRAIM DE ARAÚJO MORAIS FILHO,770338410,EFRAIM FILHO,DEM,ELEITO POR QP,EFRAIM DE ARAÚJO MORAIS FILHO,Deputado Federal,882050.00
5,5,1117690199,JOÃO HENRIQUE HOLANDA CALDAS,Deputado Federal,110000.00,NaN,NaN,NaN,NaN,NaN,...,ALAGOAS,DEPUTADO FEDERAL,JOÃO HENRIQUE HOLANDA CALDAS,1117690199,JHC,SD,ELEITO POR QP,JOÃO HENRIQUE HOLANDA CALDAS,Deputado Federal,763500.00
6,6,1266637699,NEWTON CARDOSO JÚNIOR,Deputado Federal,50000.00,NEWTON CARDOSO JÚNIOR,Deputado Federal,1601.33,NaN,NaN,...,MINAS GERAIS,DEPUTADO FEDERAL,NEWTON CARDOSO JÚNIOR,1266637699,NEWTON CARDOSO JR,PMDB,ELEITO POR QP,NEWTON CARDOSO JÚNIOR,Deputado Federal,1818364.16
7,7,1311846794,WASHINGTON REIS DE OLIVEIRA,Deputado Federal,100000.00,WASHINGTON REIS DE OLIVEIRA,Deputado Federal,500000.00,NaN,NaN,...,RIO DE JANEIRO,DEPUTADO FEDERAL,WASHINGTON REIS DE OLIVEIRA,1311846794,WASHINGTON REIS,PMDB,ELEITO POR QP,WASHINGTON REIS DE OLIVEIRA,Deputado Federal,1362633.01
8,8,1338190601,CAIO NARCIO RODRIGUES DA SILVEIRA,Deputado Federal,100000.00,CAIO NARCIO RODRIGUES DA SILVEIRA,Deputado Federal,12300.00,NaN,NaN,...,MINAS GERAIS,DEPUTADO FEDERAL,CAIO NARCIO RODRIGUES DA SILVEIRA,1338190601,CAIO NARCIO,PSDB,ELEITO POR QP,CAIO NARCIO RODRIGUES DA SILVEIRA,Deputado Federal,4053221.63
9,9,1416576770,ALESSANDRO LUCCIOLA MOLON,Deputado Federal,50000.00,NaN,NaN,NaN,NaN,NaN,...,RIO DE JANEIRO,DEPUTADO FEDERAL,ALESSANDRO LUCCIOLA MOLON,1416576770,ALESSANDRO MOLON,PT,ELEITO POR QP,ALESSANDRO LUCCIOLA MOLON,Deputado Federal,858162.50


In [107]:
soma_total_final.to_csv('dia29_11_soma_ibama.csv')

# Lista de empresas e nomes que foram doadoras - isso vai trazer todos os municípios onde há irregularidades

In [119]:
# Pega lista de CNPJs raiz e elimina duplicados

In [121]:
lista1 = pd.merge(ibama_doadores_diretos.drop_duplicates('CNPJ_raiz'), ibama_doadores_indiretos.drop_duplicates('CNPJ_raiz'), left_on='CNPJ_raiz', right_on='CNPJ_raiz', how='outer')

In [122]:
lista1.CNPJ_raiz.reset_index()

,index,CNPJ_raiz
0,0,87675518
1,1,78434982
2,2,91409698
3,3,60332319
4,4,37637139
5,5,22266175
6,6,77911261
7,7,15043391
8,8,15009178
9,9,85374635


In [123]:
# faz um merge dos dados de Ibama com a lista acima para pegar todas as linhas que têm os documentos

In [124]:
lista_ibama_1 = pd.merge(ibama_cnpjs_final, lista1, left_on='CNPJ_raiz', right_on='CNPJ_raiz')

In [125]:
lista_ibama_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 187 entries, 0 to 186
Columns: 130 entries, N_TAD to Operacoes_y
dtypes: float64(32), int64(5), object(93)
memory usage: 191.4+ KB


In [131]:
lista_ibama_1.to_csv('dia28_11_entidades_ibama.csv')

In [127]:
# Lista apenas com os CPFs

In [132]:
lista_ibama_2 = pd.merge(ibama_doadores_diretos_cpf.drop_duplicates('CPF_CNPJ_limpo'), doacoes_cpfs.drop_duplicates('CPF_CNPJ_doador'), left_on='CPF_CNPJ_limpo', right_on='CPF_CNPJ_doador')

In [133]:
lista_ibama_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40 entries, 0 to 39
Data columns (total 92 columns):
CNPJ_Prestador_Conta_x                 40 non-null int64
Sequencial_Candidato_x                 40 non-null int64
UF_x                                   40 non-null object
Partido_eleicao_x                      40 non-null object
Numero_candidato_x                     40 non-null int64
Cargo_x_x                              40 non-null object
Nome_candidato_x_x                     40 non-null object
CPF_candidato_x                        40 non-null int64
Recibo_Eleitoral_x                     40 non-null object
Numero_documento_x                     40 non-null object
CPF_CNPJ_doador_x                      40 non-null float64
Nome_doador_x                          40 non-null object
Nome_doador_Receita_x                  40 non-null object
Sigla_UE_doador_x                      40 non-null object
Numero_partido_doador_x                40 non-null object
Numero_candidato_doador_x     

In [134]:
lista_ibama_3 = pd.merge(ibama_cpfs, lista_ibama_2, left_on='CPF_CNPJ_limpo', right_on='CPF_CNPJ_limpo')

In [135]:
lista_ibama_2.to_csv('dia28_11_entidades_ibama_cpf.csv')